In [1]:
from dataset import load_dataset, load_dataloader 
from models import load_model 
import yaml 
import torch 
import numpy as np 
import random 
import os 

from accelerate import Accelerator
import matplotlib.pyplot as plt 
def img_show(img):    
    img = torch.permute(img,dims=(1,2,0)).detach().numpy()
    img = (img- np.min(img)) / (np.max(img) - np.min(img))
    #img = img[:,:,::-1]
    plt.imshow(img)
    plt.show()

def torch_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU 
    # CUDA randomness
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    


cfg = yaml.load(open('./configs/default.yaml'),Loader=yaml.Loader)

torch_seed(cfg['TRAIN']['seed'])

trainset,testset = load_dataset(
                                    dataset_name = cfg['DATA']['dataset_name'],
                                    dataset_dir  = cfg['DATA']['dataset_dir'],
                                    class_name   = cfg['DATA']['class_name'],
                                    img_size     = cfg['DATA']['img_size'],
                                    batch_size   = cfg['DATA']['batch_size'],
                                    **{'mode':'full'}
                                )

trainloader,validloader,testloader = load_dataloader(trainset,testset,testset,
                                                    batch_size = cfg['DATA']['batch_size'],
                                                    shuffle=True)

model = load_model(
                    model_name = cfg['MODEL']['model_name'],
                    pretrained = cfg['MODEL']['pretrained'],
                    num_class  = len(trainset.l2i),
                    )


# criterion 

optimizer = __import__('torch.optim', fromlist='optim').__dict__[cfg['TRAIN']['optimizer']](model.parameters(), lr=cfg['TRAIN']['lr'])

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=cfg['TRAIN']['epochs'], T_mult=1, eta_min=0.00001)

criterion = torch.nn.CrossEntropyLoss()


accelerator = Accelerator(mixed_precision = 'fp16')

# fit(
#     model = model,
#     trainloader = trainloader,
#     validloader = validloader
#     testloader = testloader,
#     optimizer = optimizer,
#     criterion = criterion,
#     config = cfg,
#     accelerator = accelerator           
# )

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [111]:
def anomaly_detection_evaluation(testloader,model):
    target_ams = [] 
    pred_ams = [] 
    target_labels = []
    model.eval() 
    for i, (imgs, gts, labels, anomaly_labels) in enumerate(testloader):
        with torch.no_grad():
            # Inference 
            outputs,(x1,x2,x3,x4) = model(imgs)
            # Average along channel 
            x2 = torch.mean(x2,dim=1,keepdim=True)
            x4 = torch.mean(x4,dim=1,keepdim=True)
            # Interpolatino to target size 
            x2 = F.interpolate(x2, size=224, mode='bilinear', align_corners=True)
            x4 = F.interpolate(x4, size=224, mode='bilinear', align_corners=True)
            # Normalize 
            x2 = torch.concat([(x - torch.min(x)) / (torch.max(x) - torch.min(x)) for x in x2])
            x4 = torch.concat([(x - torch.min(x)) / (torch.max(x) - torch.min(x)) for x in x4])
            # Calculate Anomaly Score 
            anomaly_map = torch.pow((x4 - x2),2)
            anomaly_map = torch.unsqueeze(anomaly_map,dim=1)
            # Anomaly map flatten keeping batch 
            pred_am = anomaly_map.flatten(1)
            target_am = gts.flatten(1).type(torch.int)
            
            target_ams.append(target_am.detach().cpu().numpy())
            pred_ams.append(pred_am.detach().cpu().numpy())
            
            target_labels.append(anomaly_labels.detach().cpu().numpy())
            
    target_ams = np.concatenate(target_ams)
    pred_ams = np.concatenate(pred_ams)
    
    target_labels = np.concatenate(target_labels)
    pred_labels = np.max(pred_ams,axis=1)
    
    pixel_auroc = roc_auc_score(target_ams.flatten(),pred_ams.flatten())
    img_auroc = roc_auc_score(target_labels.flatten(),pred_labels.flatten())
    return img_auroc, pixel_auroc 

In [112]:
img,pixel = anomaly_detection_evaluation(testloader,model)

In [114]:
pixel

0.7046160748302601